In [1]:
import os

# Needed for a bit more reproducibility of results when using TensorFlow
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

In [2]:
import mlflow
from experiments_config import (
    CommonConfig,
    MlflowConfig,
    DatasetConfig,
    ModelConfig,
    ModelTrainingConfig,
    ModelEvaluationConfig,
)

common_config = CommonConfig()
mlflow_config = MlflowConfig()
dataset_config = DatasetConfig()
model_config = ModelConfig()
training_config = ModelTrainingConfig()
evaluation_config = ModelEvaluationConfig()

2025-09-21 21:38:48.446043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1758483530.697083  365657 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9062 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
# log system metrics in the mlflow server
if mlflow_config.ENABLE_SYSTEM_METRICS_LOGGING:
    mlflow.enable_system_metrics_logging()

In [4]:
# Set the tracking URI and experiment for subsequent runs

mlflow.set_tracking_uri(mlflow_config.MLFLOW_TRACKING_URI)
mlflow.set_experiment(mlflow_config.MLFLOW_EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1758483410241, experiment_id='1', last_update_time=1758483410241, lifecycle_stage='active', name='RottenBot-All-Classes', tags={'dataset': 'rottenbot_all_classesv1',
 'framework': 'tensorflow-keras',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.note.content': ' This experiment focuses on detecting healthy and '
                        'rotten fruits and vegetables using computer vision. ',
 'num_classes': '28',
 'project_name': 'rotten-bot-all-classes',
 'status': 'in-progress',
 'task': 'image-classification',
 'team': 'ai-team-xyz'}>

In [5]:
# Set a custom run name for better identification in the MLflow UI
mlflow.set_tag("mlflow.runName", mlflow_config.MLFLOW_RUN_NAME)

# set the dataset as tag in the mlflow run
mlflow.set_tag("dataset", dataset_config.DATASET)

# set a description for the MLflow run
mlflow.set_tag("mlflow.note.content", mlflow_config.MLFLOW_RUN_DESCRIPTION)

In [6]:
import git

if mlflow_config.MLFLOW_LOG_GIT_SHA:
    repo = git.Repo(search_parent_directories=True)
    sha = repo.head.object.hexsha
    # set the git commit sha as a tag in the mlflow run for better traceability
    mlflow.set_tag("git_commit", sha)

In [7]:
# log the experiments_config.py for future auditability
mlflow.log_artifact(
    common_config.PATH_TO_CONFIG_FILE,
    artifact_path="config.py",
)

In [8]:
from rotten_bot.utils import get_tensorflow_dataset

# Tensorflow Dataset loading

train_dataset = get_tensorflow_dataset(
    image_folder=f"{dataset_config.DATASET_FOLDER}/train",
    image_size=dataset_config.IMAGE_SIZE,
    batch_size=dataset_config.TRAIN_BATCH_SIZE,
    label_mode=dataset_config.LABEL_MODE,
    shuffle=True,  # shuffle True for training dataset
    seed=common_config.SEED,
)

val_dataset = get_tensorflow_dataset(
    image_folder=f"{dataset_config.DATASET_FOLDER}/val",
    image_size=dataset_config.IMAGE_SIZE,
    batch_size=dataset_config.VALIDATION_BATCH_SIZE,
    label_mode=dataset_config.LABEL_MODE,
    shuffle=False,  # shuffle False for validation dataset
    seed=common_config.SEED,
)

Found 23413 files belonging to 28 classes.
Found 2922 files belonging to 28 classes.


In [9]:
import tensorflow as tf

# setup mixed precision if wanted
# Reference: https://www.tensorflow.org/guide/mixed_precision
if model_config.ENABLE_MIXED_PRECISION:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

model = model_config.MODEL

model.compile(
    optimizer=model_config.OPTIMIZER,
    loss=model_config.LOSS,
    metrics=model_config.METRICS,
)

In [10]:
from rotten_bot.utils import compute_class_weights, get_true_labels

class_weight = None

# Optionally compute class weights to handle class imbalance
if training_config.COMPUTE_CLASS_WEIGHTS:
    y_true = get_true_labels(train_dataset)
    class_weight = compute_class_weights(
        y_true, class_weight=training_config.CLASS_WEIGHTING_METHOD
    )

2025-09-21 21:38:59.659071: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-09-21 21:39:01.307902: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-09-21 21:39:02.694398: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# Actual train the model

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=training_config.EPOCHS,
    callbacks=training_config.TRAINING_CALLBACKS,
    class_weight=class_weight,
    verbose=1,
)

Epoch 1/50


2025-09-21 21:39:10.564888: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


224/366 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - f1_score: 0.6165 - loss: 1.7463

2025-09-21 21:39:22.861768: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


296/366 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - f1_score: 0.6582 - loss: 1.5606

...

366/366 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - f1_score: 0.9901 - loss: 0.0296 - val_f1_score: 0.9760 - val_loss: 0.0770 - learning_rate: 1.0000e-04
Epoch 50/50
222/366 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - f1_score: 0.9903 - loss: 0.0295

2025-09-21 21:56:25.010772: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


293/366 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - f1_score: 0.9903 - loss: 0.0293

2025-09-21 21:56:28.459745: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


366/366 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - f1_score: 0.9913 - loss: 0.0269 - val_f1_score: 0.9757 - val_loss: 0.0762 - learning_rate: 1.0000e-04


In [ ]:
# Optional log the model

# Currently not usable because Mlflow Issue

if mlflow_config.MLFLOW_LOG_MODEL:
    try:
        # get one batch of images to use as input example to infer the signature for
        # logging the model
        for x, y in train_dataset.take(1):
            input_example = x[:1].numpy()
            break

        mlflow.keras.log_model(
            model,
            **mlflow_config.MLFLOW_LOG_MODEL_CONFIG,
            input_example=input_example,
        )
    except Exception as e:
        print(f"Logging the model failed due to {e}")

In [ ]:
# Workaround for the above bug, save the model manually and log it as an artifact

if mlflow_config.MLFLOW_SAVE_MODEL_AS_ARTIFACT:
    model.save(mlflow_config.MLFLOW_SAVE_MODEL_NAME)
    mlflow.log_artifact(mlflow_config.MLFLOW_SAVE_MODEL_NAME, artifact_path="model")
    os.remove(mlflow_config.MLFLOW_SAVE_MODEL_NAME)

In [13]:
from rotten_bot.utils import (
    save_prediction_time,
    save_confusion_matrix,
    save_prediction_csv,
    save_model_history,
)
import numpy as np

# Optional evaluate the model on the test set
if evaluation_config.INCLUDE_EVALUATION_ON_TEST_SET:
    # load the test dataset
    test_dataset = get_tensorflow_dataset(
        image_folder=f"{dataset_config.DATASET_FOLDER}/test",
        image_size=dataset_config.IMAGE_SIZE,
        batch_size=dataset_config.TEST_BATCH_SIZE,
        label_mode=dataset_config.LABEL_MODE,
        shuffle=False,  # shuffle needs to be false for later evaluation
        seed=common_config.SEED,
    )

    test_results = model.evaluate(test_dataset, return_dict=True)
    # log the test results to mlflow with a "test_" prefix
    for name, value in test_results.items():
        mlflow.log_metric(f"test_{name}", value)

    if evaluation_config.SAVE_MODEL_HISTORY:
        save_model_history(history)

    # optional save the prediction time to mlflow (in milliseconds)
    if evaluation_config.SAVE_PREDICTION_TIME:
        y_probs = save_prediction_time(model, test_dataset)

    # if confusion matrix or prediction csv should be saved, we need the predicted and true labels
    # additional the file paths and class names are needed
    if evaluation_config.SAVE_CONFUSION_MATRIX or evaluation_config.SAVE_PREDICTION_CSV:
        y_pred = np.argmax(y_probs, axis=1)

        y_true = get_true_labels(test_dataset)

        file_paths = test_dataset.file_paths
        class_names = test_dataset.class_names

    # optional save the confusion matrix to mlflow as plot
    if evaluation_config.SAVE_CONFUSION_MATRIX:
        save_confusion_matrix(
            y_true,
            y_pred,
            class_names,
        )

    # optional save two csv, one with all predictions and one with missclassified samples
    if evaluation_config.SAVE_PREDICTION_CSV:
        save_prediction_csv(file_paths, y_true, y_pred, y_probs, class_names)

Found 2942 files belonging to 28 classes.
10/92 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - f1_score: 0.9949 - loss: 0.0306

2025-09-21 21:56:35.007854: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - f1_score: 0.9772 - loss: 0.0780


2025-09-21 21:56:38.878958: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-09-21 21:56:42.112983: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-09-21 21:56:43.172754: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
